##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 画像セグメンテーション

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/images/segmentation">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/images/segmentation.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/images/segmentation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/images/segmentation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

このチュートリアルでは、修正した <a href="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/" class="external">U-Net</a> を使用した画像セグメンテーションのタスクに焦点を当てます。

## 画像セグメンテーションとは

これまでに見てきた画像分類では、ネットワークの役割は入力画像にラベルやクラスを割り当てることでした。しかし、特定の物体が画像のどこにあるか、その物体の形状、どのピクセルがどの物体に属するかなどを知りたいとします。その場合は、画像をセグメント化する、すなわち画像の各ピクセルにラベルを与える必要があります。つまり、画像セグメンテーションのタスクは、ニューラルネットワークをトレーニングして画像のピクセル単位のマスクを出力することです。これは、より低いレベル、つまりピクセルレベルでの画像理解に有用です。画像セグメンテーションは、医療画像、自動運転車、衛星画像などで幅広く応用されています。

このチュートリアルで使用するデータセットは、Parkhi *et al.* によって作成された [Oxford-IIIT ペットデータセット](https://www.robots.ox.ac.uk/~vgg/data/pets/) です。このデータセットは、画像とそれに対応するラベル、ピクセル単位のマスクによって構成されています。基本的に、マスクは各ピクセルのラベルです。各ピクセルには 3 つのカテゴリのうちの 1 つが付与されます。

- クラス 1 : ペットに属するピクセル。
- クラス 2 : ペットと境界のピクセル。
- クラス 3 : 上記のいずれにも該当しない、または周囲のピクセル。

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git
!pip install -U tfds-nightly

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds

from IPython.display import clear_output
import matplotlib.pyplot as plt

## Oxford-IIIT ペットデータセットをダウンロードする

データセットは既に TensorFlow Dataset に含まれているので、これをダウンロードするだけです。セグメンテーションマスクはバージョン 3 以上に含まれています。

In [ ]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

以下のコードは、画像を反転させる単純な増強を行います。それに加えて、画像を [0,1] に正規化します。最後に、上で説明したように、セグメンテーションマスクのピクセルには {1、2、3} のいずれかのラベルが付いています。便宜上、セグメンテーションマスクから 1 を減算して、ラベルを {0、1、2} としましょう。

In [ ]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask -= 1
  return input_image, input_mask

In [ ]:
@tf.function
def load_image_train(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  if tf.random.uniform(()) &gt; 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_mask)

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [ ]:
def load_image_test(datapoint):
  input_image = tf.image.resize(datapoint['image'], (128, 128))
  input_mask = tf.image.resize(datapoint['segmentation_mask'], (128, 128))

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

データセットには、既にテストとトレーニングに必要な分割が含まれているため、同じ分割を使用して続けましょう。

In [ ]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = dataset['test'].map(load_image_test)

In [ ]:
train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

画像の例とデータセットに対応するマスクを表示してみましょう。

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
for image, mask in train.take(1):
  sample_image, sample_mask = image, mask
display([sample_image, sample_mask])

## モデルを定義する

ここで使用するモデルは、修正された U-Net です。U-Net はエンコーダ（ダウンサンプラ）とデコーダ（アップサンプラ）で構成されています。ロバストな特徴を学習し、トレーニング可能なパラメータ数を減らすため、事前トレーニング済みモデルをエンコーダとして使用することができます。したがって、このタスクのエンコーダは事前トレーニング済み MobileNetV2 モデルで、その中間出力を使用します。また、デコーダは <a class="" href="https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py">Pix2pix チュートリアル</a>の TensorFlow Examples に既に実装されているアップサンプルブロックです。

3 つのチャンネルを出力する理由は、各ピクセルには 3 つの可能性のあるラベルがあるからです。これは、マルチ分類で各ピクセルが 3 つのクラスに分類されると考えるとよいでしょう。

In [ ]:
OUTPUT_CHANNELS = 3

前述したように、エンコーダは事前トレーニング済み MobileNetV2 モデルであり、[tf.keras.applications](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/applications) で即座に使用できるように準備されています。エンコーダはモデル内の中間レイヤーからの特定の出力で構成されています。トレーニングプロセス中にエンコーダはトレーニングされないので注意してください。

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

デコーダおよびアップサンプラは、単に TensorFlow の 例に実装されている一連のアップサンプラブロックに過ぎません。

In [ ]:
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -&gt; 8x8
    pix2pix.upsample(256, 3),  # 8x8 -&gt; 16x16
    pix2pix.upsample(128, 3),  # 16x16 -&gt; 32x32
    pix2pix.upsample(64, 3),   # 32x32 -&gt; 64x64
]

In [ ]:
def unet_model(output_channels):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # Downsampling through the model
  skips = down_stack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same')  #64x64 -&gt; 128x128

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

## モデルをトレーニングする

あとはモデルをコンパイルしてトレーニングするのみです。ここで使用する損失は、`losses.SparseCategoricalCrossentropy(from_logits=True)`です。ネットワークがマルチクラス予測のように各ピクセルにラベルの割り当てを試みるため、この損失関数を使用します。真のセグメンテーションマスクでは、各ピクセルは {0、1、2} のいずれかのマスクを持ちます。ネットワークはここで 3 つのチャンネルを出力します。原則的に各チャンネルはクラスを予測するための学習を試みるので、`losses.SparseCategoricalCrossentropy(from_logits=True)`は、このようなシナリオに推奨される損失です。ネットワークの出力を使用して、ピクセルに割り当てられるラベルは最も高い値を持つチャンネルになります。create_mask 関数がそれを行います。

In [ ]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

結果のモデルアーキテクチャを簡単に見てみましょう。

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

トレーニングする前に、モデルが何を予測するかを試して表示します。

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
show_predictions()

トレーニング中にモデルがどのように改善されるかを観察してみましょう。このタスクを遂行するために、コールバック関数を以下のように定義します。 

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [ ]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_dataset, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_dataset,
                          callbacks=[DisplayCallback()])

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(EPOCHS)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

## 予測する

いくつか予測を行ってみましょう。時間の節約重視の場合はエポック数を少なくしますが、高精度の結果重視の場合はエポック数を増やして設定します。

In [ ]:
show_predictions(test_dataset, 3)

## 次のステップ

これで画像セグメンテーションとは何か、それがどのように機能するかについての知識が得られたはずです。このチュートリアルは、異なる中間レイヤー出力や、異なる事前トレーニング済みモデルでも試すことができます。また、Kaggle がホストしている [Carvana](https://www.kaggle.com/c/carvana-image-masking-challenge/overview) 画像マスキングチャレンジに挑戦してみることもお勧めです。

[Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) を参照して、独自のデータで再トレーニング可能な別のモデルを確認するのも良いでしょう。